In [2]:
import os
from langchain_community.document_loaders import PyPDFLoader

def load_resume(file_path):
    if not os.path.exists(file_path):
        raise FileNotFoundError(f"The file {file_path} does not exist.")
    
    loader = PyPDFLoader(file_path)
    documents = loader.load()
    return documents

d:\The-Office\Projects\RAG\cold-email\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
import pandas as pd

def load_jobs(path="data/jobs.csv"):
    return pd.read_csv(path)

In [4]:
def get_resume_text(docs):
    return "\n".join(d.page_content for d in docs)

In [5]:
from sentence_transformers import SentenceTransformer, util
model = SentenceTransformer('all-MiniLM-L6-v2')

def rank_jobs_for_resume(jobs_df, resume_text):

    resume_emb = model.encode(resume_text, convert_to_tensor=True)
    job_embs = model.encode(jobs_df["job_description"].tolist(), convert_to_tensor=True)

    cos_scores = util.cos_sim(resume_emb, job_embs)[0]

    all_scores = cos_scores.cpu().numpy()

    ranked_jobs_df = jobs_df.copy()
    ranked_jobs_df["similarity_score"] = all_scores
    sorted_jobs_df = ranked_jobs_df.sort_values(by="similarity_score", ascending=False).reset_index(drop=True)

    return sorted_jobs_df

In [6]:
from groq import Groq
import os
from dotenv import load_dotenv

load_dotenv()
client = Groq(api_key=os.getenv("GROQ_API_KEY"))

def generate_email(resume, job):
    prompt = f"""
Write a short, sharp cold email (<120 words) applying for this role: {job['job_title']}.

My resume summary:
{resume}

Job description:
{job['job_description']}

Email must:
- be 5–6 sentences
- highlight relevant skills
- sound confident and professional
- end with a call to action
"""

    response = client.chat.completions.create(
        model="llama-3.3-70b-versatile",
        messages=[{"role": "user", "content": prompt}]
    )

    return response.choices[0].message.content

In [ ]:
import yagmail

def send_email(to, subject, body, attachment=None):
    EMAIL = os.getenv("EMAIL")
    APP_PASSWORD = os.getenv("APP_PASSWORD")
    yag = yagmail.SMTP(EMAIL,APP_PASSWORD)

    yag.send(to=to, subject=subject, contents=body, attachments=attachment)

In [7]:
resume_docs = load_resume("../data/resume.pdf")
resume_text = get_resume_text(resume_docs)
jobs = load_jobs("../data/jobs.csv")

jobs_ranked = rank_jobs_for_resume(jobs, resume_text)


In [8]:
jobs_ranked[["job_title", "similarity_score"]].head(5)

,job_title,similarity_score
0,Machine Learning Intern,0.430860
1,Data Analyst Intern,0.391007
2,NLP Intern,0.369782
3,AI Research Intern,0.355409


In [9]:
email_content = generate_email(resume_text, jobs_ranked.iloc[0])
print(email_content)

Subject: Application for Machine Learning Intern

Dear Hiring Manager,

I'm excited to apply for the Machine Learning Intern role, leveraging my strong foundation in machine learning, data preprocessing, and model evaluation. With a Master's in Data Science from Vellore Institute of Technology, I've developed expertise in Python, PyTorch, and scikit-learn, with a proven track record of delivering AI/ML solutions for clients. My experience with data visualization tools like Power BI and SQL has enabled me to drive business insights and inform decision-making. I'm confident in my ability to design and implement classical ML and deep learning models, and I'm eager to bring my skills to your team. I'd love to discuss my application and how I can contribute to your organization. Can we schedule a call to explore further?

Best regards,
Vainavi Nair


In [11]:
from dotenv import load_dotenv
load_dotenv()
TEST_EMAIL = os.getenv("TEST_EMAIL")
attachment_path = "../data/resume_x.pdf"
send = input("Send email? (y/n): ")
if send.lower() == 'y':
    send_email(
        #to=job['contact_email'],
        to=TEST_EMAIL,
        subject=f"Application for { jobs_ranked.iloc[0]['job_title']}",
        body=email_content,
        attachment=attachment_path
    )
    print("Email sent!")

Email sent!
